In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import ast

In [ ]:
users_df = pd.read_csv('./output/users.csv')
repos_df = pd.read_csv('./output/repos.csv')

In [ ]:
repo_occurrence_dict = defaultdict(int)
for _, user in users_df.iterrows():
    repos = ast.literal_eval(user['repos'])
    for repo in repos:
        repo_occurrence_dict[repo] += 1

collaborated_repos = set([key for key, val in repo_occurrence_dict.items() if val > 1])

In [ ]:
offset = 7201
limit = 1000

users_df_slice = users_df.iloc[offset : offset + limit]
users_df_slice

In [ ]:
fetched_commits_data = []

for index, user in users_df_slice.iterrows():
    try:
        print(f"Searching user #{index + 1}: {user['login']}")
        repos = [repo for repo in ast.literal_eval(user['repos']) if repo in collaborated_repos]
    except Exception as e:
        print(e)

    for index, repo in enumerate(repos):
        print(f"Searching repo {index + 1} of {len(repos)}: {repo}")
        try:
            while g.rate_limiting[0] < 50:
                print("rate limit almost approached, staling...")
                sleep(10)
                g.get_rate_limit()
            
            # repo_obj = g.get_repo(repo)
            # print(repo_obj.get_commits(author=user["login"]).raw_data)
            cur_commit_objs = g.get_repo(repo).get_commits(author=user["login"])
            
            cur_commit_objs[0]
            
            # print(cur_commit_objs.totalCount)
            cur_commit_data = []

            if (cur_commit_objs.totalCount < 100000):
                for commit_obj in cur_commit_objs:
                    date = commit_obj.commit.author.date.isoformat()
                    cur_commit_data.append(date)
                    print(cur_commit_objs.totalCount, date, end="\r")
            else:
                print('Too much commits, fetching by months')
                cur_start = cur_commit_objs[0].commit.author.date
                cur_end = cur_commit_objs.reversed[0].commit.author.date
                pr = pd.period_range(start="2013-05", end=f"2023-05", freq="M")
                print(cur_start, cur_end, pr)
                for p in pr:
                    try:
                        cur_period_commit_objs = g.get_repo(repo).get_commits(author=user["login"], since=p.start_time.to_pydatetime(), until=p.end_time.to_pydatetime())
                        cur_commit_objs[0]
                        if cur_period_commit_objs.totalCount > 0:
                            cur_commit_data.append(cur_period_commit_objs[0].commit.author.date.isoformat())
                        print(p, cur_period_commit_objs.totalCount, end="\r")
                    except:
                        pass


            fetched_commits_data.append({
                "user": user['login'],
                "repo": repo,
                "commits": cur_commit_data
            })
        except Exception as e:
            print(e)


In [ ]:
page_index = 7
pd.DataFrame.from_records(fetched_commits_data).to_csv(f'./output/commits-{page_index}.csv')